In [ ]:
!pip install -r requirements.txt


# 1 Connexion à PostgreSQL


In [25]:
import psycopg

# Paramètres de connexion
db_params = {
    "host": "localhost",
    "port": 5432,
    "dbname": "rag_chatbot",
    "user": "postgres",
    "password": "aya"
}

conn = psycopg.connect(**db_params)
cur = conn.cursor()
print("✅ Connexion à la base de données réussie")



✅ Connexion à la base de données réussie


# 2 Création de la table pour le corpus + embeddings

In [8]:

cur.execute("""
CREATE TABLE IF NOT EXISTS documents (
    id SERIAL PRIMARY KEY,
    corpus TEXT NOT NULL,
    embedding vector(1536) -- taille embedding MiniLM par défaut
);
""")
conn.commit()
print("✅ Table 'documents' créée")

# Si pgvector n'est pas activé :
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
conn.commit()
print("✅ Extension pgvector activée")


✅ Table 'documents' créée
✅ Extension pgvector activée


# 3 import du corpus

In [11]:
import os

data_folder = "../data/"
corpus_data = []

for file_name in os.listdir(data_folder):
    if file_name.endswith(".txt"):
        file_path = os.path.join(data_folder, file_name)

        try:
            with open(file_path, "r", encoding="utf-8") as f:
                lines = f.readlines()

        except UnicodeDecodeError:
            print(f"⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : {file_name}")
            with open(file_path, "r", encoding="latin-1") as f:
                lines = f.readlines()

        corpus_data.extend(lines)
        print(f"📄 Chargé : {file_name} → {len(lines)} lignes")

print(f"\n📌 Total des lignes dans le corpus = {len(corpus_data)}")


⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 017_00000012.txt
📄 Chargé : 017_00000012.txt → 78 lignes
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 018_00000013.txt
📄 Chargé : 018_00000013.txt → 17 lignes
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 019_00000014.txt
📄 Chargé : 019_00000014.txt → 23 lignes
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 020_00000015.txt
📄 Chargé : 020_00000015.txt → 25 lignes
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 022_00000017.txt
📄 Chargé : 022_00000017.txt → 43 lignes
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 023_00000018.txt
📄 Chargé : 023_00000018.txt → 27 lignes
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 024_00000019.txt
📄 Chargé : 024_00000019.txt → 190 lignes
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 027_0000001c.txt
📄 Chargé : 027_0000001c.txt → 63 lignes
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 028_0000001d.txt
📄 Chargé : 028_0000001d.txt → 197 lignes
⚠️ Encodage non-U

# Nettoyer le corpus


In [12]:
import os
import re

data_folder = "../data/"
clean_corpus = []

# Parcours de tous les fichiers .txt
for file_name in os.listdir(data_folder):
    if file_name.endswith(".txt"):
        file_path = os.path.join(data_folder, file_name)
        
        # Lecture avec gestion d'encodage
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                lines = f.readlines()
        except UnicodeDecodeError:
            print(f"⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : {file_name}")
            with open(file_path, "r", encoding="latin-1") as f:
                lines = f.readlines()
        
        # Nettoyage ligne par ligne
        for line in lines:
            line = line.strip()  # enlever les espaces au début et à la fin
            if not line:
                continue  # ignorer lignes vides
            # Supprimer les numéros <01> etc.
            line = re.sub(r"<\d+>", "", line)
            # Supprimer les identifiants "h:" ou "c:" si besoin ou les garder
            line = re.sub(r"^[hc]:\s*", "", line)
            # Supprimer les caractères non imprimables ou spéciaux
            line = re.sub(r"[^\x00-\x7F]+", " ", line)
            # Supprimer espaces multiples
            line = re.sub(r"\s+", " ", line)
            clean_corpus.append(line.strip())

print(f"✨ Total lignes nettoyées : {len(clean_corpus)}")
# Afficher les premières lignes pour vérification
for i, line in enumerate(clean_corpus[:10]):
    print(f"{i+1}: {line}")



⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 017_00000012.txt
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 018_00000013.txt
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 019_00000014.txt
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 020_00000015.txt
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 022_00000017.txt
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 023_00000018.txt
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 024_00000019.txt
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 027_0000001c.txt
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 028_0000001d.txt
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 029_0000001e.txt
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 030_0000001f.txt
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 037_00000026.txt
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 038_00000027.txt
⚠️ Encodage non-UTF8 détecté → Lecture en latin-1 : 045_0000002e.txt
⚠️ Encodage non-UTF8 détecté → Lec

# créer les embeddings

In [15]:
from sentence_transformers import SentenceTransformer
import psycopg
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Modèle SentenceTransformer chargé")


c:\Users\ayate\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Modèle SentenceTransformer chargé


In [26]:
import psycopg

batch_size = 50  # traitement par lots
total_inserted = 0

cur = conn.cursor()

for i in range(0, len(clean_corpus), batch_size):
    batch = clean_corpus[i:i+batch_size]
    for line in batch:
        emb = model.encode(line)            # Générer l'embedding
        emb_list = emb.tolist()             # Convertir en liste pour PostgreSQL

        try:
            # INSERT dans la table avec pgvector
            cur.execute(
                "INSERT INTO documents (text, embedding) VALUES (%s, %s);",
                (line, emb_list)
            )
        except Exception as e:
            print(f"⚠️ Erreur insertion ligne : {e}")
            conn.rollback()  # rollback si erreur
        else:
            conn.commit()
            total_inserted += 1


print(f"✅ {total_inserted} lignes insérées avec embeddings")


✅ 1982 lignes insérées avec embeddings


In [27]:
import psycopg

cur = conn.cursor()
# Vérification : récupérer 5 premières lignes insérées
cur.execute("SELECT id, text FROM documents LIMIT 5;")
rows = cur.fetchall()

print("📌 5 premières lignes de la table documents :")
for r in rows:
    print(r)

📌 5 premières lignes de la table documents :
(1, 'hotesse')
(2, 'U B S bonjour')
(3, 'client')
(4, "oui bonjour e j'appelle je sais pas si j'appelle au bon endroit e")
(5, 'hotesse+client')


# Préparer une fonction simple pour la recherche RAG (recherche par similarité)

In [42]:
pip install python-dotenv --timeout=200


Note: you may need to restart the kernel to use updated packages.


# Utiliser la clé API avec ton client HTTP

In [43]:
from groq import Groq
import os
from dotenv import load_dotenv

# Charger le .env
load_dotenv(dotenv_path="../src/.env")  # adapte le chemin si nécessaire

# Récupérer la clé API
api_key = os.getenv("GROK_API_KEY")
if not api_key:
    raise ValueError("❌ GROQ_API_KEY n'est pas défini dans .env")

# Initialiser le client Groq
client = Groq(api_key=api_key)

# Envoyer une question
response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": "Quels stages sont proposés en anglais cet été ?"}],
    max_tokens=300,
    temperature=0.2
)

print("💬 Réponse Grok :", response.choices[0].message.content)


💬 Réponse Grok : Il existe de nombreux stages proposés en anglais pendant l'été, allant des programmes universitaires aux activités de loisirs en passant par les formations professionnelles. Voici quelques exemples de stages qui pourraient vous intéresser :

1. **Programmes universitaires d'été** : De nombreuses universités à travers le monde, comme Oxford, Cambridge, Harvard, etc., proposent des programmes d'été en anglais pour les étudiants internationaux. Ces programmes couvrent une large gamme de sujets, de la littérature à la science en passant par les affaires.

2. **Stages de langue** : Si vous souhaitez améliorer votre anglais, des stages de langue sont proposés dans différents pays anglophones. Ces stages combinent souvent des cours d'anglais avec des activités culturelles et des excursions.

3. **Stages professionnels** : Des entreprises et des organisations dans divers secteurs (technologie, finance, santé, etc.) offrent des stages en anglais pour les étudiants et les jeunes

# Envoyer la question + contexte à Groq pour générer une réponse

In [45]:
# rag_agent/rag_pipeline.py

import os
import psycopg
from groq import Groq
from dotenv import load_dotenv

# -----------------------------
# Charger le .env et clé API
# -----------------------------
load_dotenv(dotenv_path="../src/.env")
GROQ_API_KEY = os.getenv("GROK_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY n'est pas défini dans .env")

client = Groq(api_key=GROQ_API_KEY)

cur = conn.cursor()

# -----------------------------
# Fonction pour récupérer les chunks les plus proches
# -----------------------------
def get_relevant_chunks(query, top_k=5):
    """
    Recherche les top_k chunks les plus similaires à la question via pgvector
    """
    # Générer l'embedding de la question avec le même modèle que celui utilisé pour le corpus
    query_emb = model.encode(query).tolist()  # <-- ici ton modèle d'embeddings

    cur.execute("""
        SELECT text, 1 - (embedding <#> %s::vector) AS similarity
        FROM documents
        ORDER BY embedding <#> %s::vector
        LIMIT %s
    """, (query_emb, query_emb, top_k))
    
    results = cur.fetchall()
    # Chaque chunk est un dictionnaire {"text": ..., "score": ...}
    chunks = [{"text": r[0], "score": r[1]} for r in results]
    return chunks

# -----------------------------
# Fonction RAG : recherche + LLM
# -----------------------------
def rag_answer(question, top_k=5, model_name="llama-3.3-70b-versatile"):
    # 1️⃣ Récupérer les chunks pertinents
    chunks = get_relevant_chunks(question, top_k=top_k)
    
    # 2️⃣ Construire le contexte
    context = "\n---\n".join([c["text"] for c in chunks])
    prompt = f"Question: {question}\nContext:\n{context}"
    
    # 3️⃣ Envoyer à Groq LLM
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=300,
        temperature=0.2
    )
    
    # 4️⃣ Retourner la réponse
    return response.choices[0].message.content

# -----------------------------
# Exemple d'utilisation
# -----------------------------
if __name__ == "__main__":
    question = "Quels stages sont proposés en anglais cet été ?"
    answer = rag_answer(question)
    print("💬 Réponse RAG :", answer)

    cur.close()
    conn.close()


💬 Réponse RAG : Il semble que vous cherchiez des informations sur les stages proposés en anglais cet été, mais les détails fournis dans le contexte ne sont pas clairs ou directs. Cependant, je peux vous offrir une approche générale pour trouver des stages en anglais pendant l'été :

1. **Recherche en ligne** : Utilisez des moteurs de recherche pour trouver des stages en anglais. Vous pouvez utiliser des mots-clés comme "summer internships in English", "English language summer programs", ou "international summer internships" pour trouver des opportunités qui correspondent à vos intérêts et à vos compétences.

2. **Sites web d'offres d'emploi et de stages** : De nombreux sites web proposent des offres de stages et d'emplois, y compris des opportunités en anglais. Vous pouvez visiter des sites comme Indeed, LinkedIn, ou Glassdoor pour trouver des stages en anglais.

3. **Universités et écoles de langues** : Les universités et les écoles de langues, comme celle que vous mentionnez à Lorien